In [1]:
import glob

In [2]:
import pandas as pd

In [3]:
from intervaltree import Interval, IntervalTree

In [4]:
target_seq = 'F11.fasta'

In [5]:
target_true_embl = 'F11.original.embl'

In [6]:
mode = 'Strain'

## Utils

In [7]:
def save_as_coord(df, file):
    with open(file, 'w') as file:
        for row in df.itertuples():
            line = '{:05}'.format(row.id) + ' ' + '{: >7}'.format(row.start) + ' ' + '{: >7}'.format(row.end)
            file.write(line + '\n')

## True Target Genes

In [8]:
!python embl_locus.py $target_true_embl locus > target_true_genes.coords

In [9]:
target_true_genes = pd.read_csv('target_true_genes.coords', header=None, delim_whitespace=True)
target_true_genes.columns = ['id', 'start', 'end', 'locus_tag']

In [10]:
target_true_genes.head()

,id,start,end,locus_tag
0,1,102,1625,TBFG_10001
1,2,2153,3361,TBFG_10002
2,3,3381,4538,TBFG_10003
3,4,4535,5098,TBFG_10004
4,5,5224,7368,TBFG_10005


In [11]:
target_true_genes.shape

(3991, 4)

### Minimum Gene Length Actually

In [12]:
min(abs(target_true_genes.end - target_true_genes.start))

70

## RATT

In [13]:
ratt_command = '$RATT_HOME/start.ratt.sh'

In [14]:
!$ratt_command embl $target_seq Result $mode

Of the reference chromosome Tb_H37Rv	2.66 per cent	has no synteny with the query
Of the query chromosome gi_148821191_ref_NC_009565.1_	 3.03 per cent	has no synteny with the reference
Nucmer is done. Now transfer the annotation.
perl /home/shonku/ratt/ratt-code/main.ratt.pl embl nucmer.Result.snp nucmer.Result.filter.coords Result
working on Tb_H37Rv
Overview of transfere of annotation elements:
9557	elements found.
9294	Elements were transfered.
0	Elements could be transfered partially.
0	Elements split.
263	Parts of elements (i.e.exons tRNA) not transferred.
263	Elements couldn't be transferred.

CDS:
3999	Gene models to transfer.
3898	Gene models transferred correctly.
0	Gene models partially transferred.
0	Exons not transferred from partial CDS matches.
101	Gene models not transferred.

1 Sequences where generated
Done.
Nucmer is done. Now Correct the annotation for chromosome .
work on gi_148821191_ref_NC_009565.1_
************************ Correction *****
Using the default specif

In [15]:
ratt_final = glob.glob('*.final.embl')[0]

In [16]:
ref_embl = glob.glob('embl/*.embl')[0]

In [17]:
!python embl_locus.py $ratt_final locus > target_ratt_genes.coords

In [18]:
target_ratt_genes = pd.read_csv('target_ratt_genes.coords', header=None, delim_whitespace=True)
target_ratt_genes.columns = ['id', 'start', 'end', 'locus_tag']

In [19]:
target_ratt_genes.head()

,id,start,end,locus_tag
0,1,102,1625,Rv0001
1,2,2153,3361,Rv0002
2,3,3381,4538,Rv0003
3,4,4535,5098,Rv0004
4,5,5224,7368,Rv0005


In [20]:
target_ratt_genes.shape

(3898, 4)

### Genes Left to Discover

In [21]:
target_true_genes.shape[0] - target_ratt_genes.shape[0]

93

## Exclude Not Transferred Genes

### Extracting Genes from Reference

In [34]:
!python embl_locus.py $ref_embl locus > ref_genes.coords

In [35]:
ref_genes = pd.read_csv('ref_genes.coords', header=None, delim_whitespace=True)
ref_genes.columns = ['id', 'start', 'end', 'locus_tag']

In [36]:
ref_genes.shape

(3999, 4)

In [37]:
ref_genes.head()

,id,start,end,locus_tag
0,1,1,1524,Rv0001
1,2,2052,3260,Rv0002
2,3,3280,4437,Rv0003
3,4,4434,4997,Rv0004
4,5,5123,7267,Rv0005


### Keeping only the locus tags that are not transferred

In [40]:
transferred_locus_tags = set(target_ratt_genes.locus_tag)

In [43]:
ref_left_genes_list = []
for row in ref_genes.itertuples():
    if row.locus_tag not in transferred_locus_tags:
        ref_left_genes_list.append(tuple(row[1:]))

In [44]:
ref_left_genes = pd.DataFrame(ref_left_genes_list, columns=['id', 'start', 'end', 'locus_tag'])

In [45]:
ref_left_genes.shape

(101, 4)

In [46]:
ref_left_genes.head()

,id,start,end,locus_tag
0,1345,1484217,1482514,Rv1320c
1,1346,1484279,1484959,Rv1321
2,1347,1484982,1485278,Rv1322
3,1348,1485771,1485313,Rv1322A
4,1349,1485862,1487031,Rv1323


In [47]:
save_as_coord(ref_left_genes, 'ref_left_genes.coords')

### Generating Multi Fasta

In [49]:
ref_seq = glob.glob('Reference.*.fasta')[0]

In [51]:
!extract -t $ref_seq ref_left_genes.coords > ref_left_genes.fasta

## Glimmer long-orfs Output

Long orf is operated on default minimum length.

This has to be changed.

In [22]:
!long-orfs -n -t 1.15 $target_seq target_orfs.coords

Starting at Wed Jan 15 21:26:34 2020

Sequence file = F11.fasta
Excluded regions file = none
Circular genome = true
Initial minimum gene length = 90 bp
Determine optimal min gene length to maximize number of genes
Maximum overlap bases = 30
Start codons = atg,gtg,ttg
Stop codons = taa,tag,tga
Sequence length = 4424435
Final minimum gene length = 727
Number of genes = 1565
Total bases = 2037894


In [24]:
target_orfs = pd.read_csv('target_orfs.coords', header=None, delim_whitespace=True)

In [25]:
target_orfs.columns = ['id', 'start', 'end', 'dir', 'score']

In [26]:
target_orfs.head()

,id,start,end,dir,score
0,1,102,1625,3,0.729
1,2,2153,3361,2,0.719
2,3,3381,4538,3,0.935
3,4,5224,7368,1,0.638
4,5,7403,9919,2,0.823


In [27]:
target_orfs.shape

(1565, 5)

## Find potential New Genes by Excluding RATT discovered ones

For each from `target_orfs`, if it has some overlap on any subseqs of `target_ratt_genes`, discard it.

In [107]:
tree = IntervalTree()

In [108]:
for row in target_ratt_genes.itertuples():
    s, e = row.start, row.end
    if s > e: s, e = e, s
    tree.add(Interval(s, e, row.id))

In [109]:
target_potn_genes_list = []
Id = 1
for row in target_orfs.itertuples():
    if len(tree[row.start]) > 0: continue
    if len(tree[row.end]) > 0: continue
    if len(tree[row.start:row.end]) > 0: continue
    if len(tree[row.end:row.start]) > 0: continue
    target_potn_genes_list.append((row.id, row.start, row.end))
    Id += 1

In [110]:
target_potn_genes = pd.DataFrame(target_potn_genes_list, columns=['id', 'start', 'end'])

In [111]:
target_potn_genes.head()

,id,start,end
0,10,26982,26014
1,231,661546,662568
2,286,854596,855369
3,332,944993,946282
4,366,1030642,1029323


In [112]:
target_potn_genes.shape

(49, 3)

In [113]:
save_as_coord(target_potn_genes, 'target_potn_genes.coords')

In [114]:
!extract -t $target_seq target_potn_genes.coords > target_potn_genes.fasta

## Blast

In [52]:
!makeblastdb -in ref_left_genes.fasta -out refleftblastdb -parse_seqids -dbtype nucl



Building a new DB, current time: 01/15/2020 21:39:29
New DB name:   /home/shonku/Project/Tb/refleftblastdb
New DB title:  ref_left_genes.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 101 sequences in 0.00926709 seconds.




In [115]:
!blastn -outfmt 6 -db refleftblastdb -query target_potn_genes.fasta -out refleftblasts.out

In [116]:
refleftblasts = pd.read_csv('refleftblasts.out', header=None, delim_whitespace=True)

In [117]:
refleftblasts.columns = [
    'qseqid', 'sseqid', 'pident', 'length',
    'mismatch', 'gapopen', 'qstart', 'qend',
    'sstart', 'send', 'evalue', 'bitscore'
]

In [118]:
refleftblasts.shape

(37, 12)

### Remove Duplicate Mapping Keeping the High Scored Ones

In [119]:
refleftblasts.sort_values(by=['bitscore'], ascending=False, inplace=True)

In [120]:
refleftblasts.drop_duplicates(subset='qseqid', keep='first', inplace=True)

In [121]:
refleftblasts.drop_duplicates(subset='sseqid', keep='first', inplace=True)

In [122]:
refleftblasts.shape

(26, 12)

In [123]:
refleftblasts.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
20,1311,3362,100.0,4539,0,0,1,4539,1,4539,0.0,8383.0
3,550,1354,100.0,2589,0,0,1,2589,1,2589,0.0,4782.0
1,548,1352,100.0,2193,0,0,1,2193,1,2193,0.0,4050.0
2,549,1353,100.0,2103,0,0,109,2211,1,2103,0.0,3884.0
4,551,1355,100.0,1992,0,0,1,1992,1,1992,0.0,3679.0


## Annotation by Matching ID

`qseqid` matched from `target_orfs`.

`sseqid` matched from `ref_genes`.

In [124]:
target_orfs.head()

,id,start,end,dir,score
0,1,102,1625,3,0.729
1,2,2153,3361,2,0.719
2,3,3381,4538,3,0.935
3,4,5224,7368,1,0.638
4,5,7403,9919,2,0.823


In [125]:
ref_genes.head()

,id,start,end,locus_tag
0,1,1,1524,Rv0001
1,2,2052,3260,Rv0002
2,3,3280,4437,Rv0003
3,4,4434,4997,Rv0004
4,5,5123,7267,Rv0005


In [126]:
fast_ref_genes = ref_genes.set_index('id')
fast_target_orfs = target_orfs.set_index('id')

In [137]:
target_new_genes_list = []
count = 1
for row in refleftblasts.itertuples():
    start, end = fast_target_orfs.loc[row.qseqid, ['start', 'end']]
    Id = 'n' + str(count)
    target_new_genes_list.append((Id, int(start), int(end), fast_ref_genes.loc[row.sseqid].locus_tag))
    count += 1

In [138]:
target_new_genes = pd.DataFrame(target_new_genes_list, columns=['id', 'start', 'end', 'locus_tag'])

In [140]:
target_new_genes.head()

,id,start,end,locus_tag
0,n1,3688490,3693031,Rv3296
1,n2,1501362,1503953,Rv1328
2,n3,1499110,1496915,Rv1326c
3,n4,1501331,1499118,Rv1327c
4,n5,1505987,1503993,Rv1329c


In [194]:
target_new_genes.to_csv('target_new_genes.csv', index=False)

## Proof of Concept

In [167]:
target_true_genes.loc[(target_true_genes.start >= 3688490) & (target_true_genes.end <= 3693031)]

,id,start,end,locus_tag
3349,3350,3688490,3693031,TBFG_13325


In [168]:
target_ratt_genes.loc[(target_ratt_genes.start >= 3688000) & (target_ratt_genes.end <= 3693100)]

,id,start,end,locus_tag


## Evaluate New Genes

In [158]:
def interval_overlap(a, b):
    return max(0, min(a[1], b[1]) - max(a[0], b[0]))

In [193]:
buffer = 300
total_overlap = 0
count = 0
for row in target_new_genes.itertuples():
    count += 1
    start1, start2 = row.start - buffer, row.start + buffer
    end1, end2 = row.end - buffer, row.end + buffer
    matchs = target_true_genes.loc[(start1 <= target_true_genes.start) & 
                                   (target_true_genes.start <= start2) &
                                   (end1 <= target_true_genes.end) & 
                                   (target_true_genes.end <= end2)]
    
    if len(matchs) == 0: continue
    if len(matchs) > 1: print(row.id)
    match = matchs.iloc[0]
    
    true_length = abs(match.start - match.end)
    pred = sorted([row.start, row.end])
    true = sorted([match.start, match.end])
    
    overlap = interval_overlap(pred, true)
    
    total_overlap += (overlap / true_length)
    
    print(tuple(match))
    print(tuple(row[1:]))
    print('--')

print('mean overlap:', total_overlap / count)
print('new transfer:', count)

(3350, 3688490, 3693031, 'TBFG_13325')
('n1', 3688490, 3693031, 'Rv3296')
--
(1381, 1501362, 1503953, 'TBFG_11360')
('n2', 1501362, 1503953, 'Rv1328')
--
(1379, 1499110, 1496915, 'TBFG_11358')
('n3', 1499110, 1496915, 'Rv1326c')
--
(1380, 1501223, 1499118, 'TBFG_11359')
('n4', 1501331, 1499118, 'Rv1327c')
--
(1382, 1505987, 1503993, 'TBFG_11361')
('n5', 1505987, 1503993, 'Rv1329c')
--
(3339, 3678072, 3679874, 'TBFG_13314')
('n6', 3678072, 3679874, 'Rv3285')
--
(3356, 3701157, 3699400, 'TBFG_13331')
('n7', 3701157, 3699400, 'Rv3302c')
--
(3468, 3840019, 3838283, 'TBFG_13443')
('n8', 3840019, 3838283, 'Rv3409c')
--
(1372, 1491016, 1489313, 'TBFG_11351')
('n9', 1491016, 1489313, 'Rv1320c')
--
(3463, 3835155, 3833554, 'TBFG_13438')
('n10', 3835266, 3833554, 'Rv3403c')
--
(3462, 3833183, 3831945, 'TBFG_13437')
('n11', 3833198, 3831945, 'Rv3402c')
--
(3482, 3852712, 3851486, 'TBFG_13457')
('n12', 3852712, 3851486, 'Rv3423c')
--
(3485, 3857259, 3855175, 'TBFG_13460')
('n13', 3857259, 3855175,

## Combined

In [191]:
target_final_genes = pd.concat([target_ratt_genes, target_new_genes], axis=0)

In [192]:
target_final_genes.shape

(3924, 4)

In [195]:
target_final_genes.to_csv('target_final_genes.csv', index=False)